In [31]:
import pandas as pd
import numpy as np
import json
import hashlib
import collections
import re
from datetime import datetime
from datetime import timedelta
from collections import OrderedDict

In [32]:
dataEvents = pd.read_csv('/Users/manuelgomezmoratilla/Desktop/TFG/data_processing_scripts/data/anonamyze_all_data_collection_v2.csv', sep=";")

In [33]:
pd.options.mode.chained_assignment = None  # default='warn'
def sequenceWithinPuzzles(dataEvents, group = 'all'):
    
    dataEvents['group'] = [json.loads(x)['group'] if 'group' in json.loads(x).keys() else '' for x in dataEvents['data']]
    dataEvents['user'] = [json.loads(x)['user'] if 'user' in json.loads(x).keys() else '' for x in dataEvents['data']]
    # removing those rows where we dont have a group and a user that is not guest
    dataEvents = dataEvents[((dataEvents['group'] != '') & (dataEvents['user'] != '') & (dataEvents['user'] != 'guest'))]
    dataEvents['group_user_id'] = dataEvents['group'] + '~' + dataEvents['user']
    # filtering to only take the group passed as argument
    if(group != 'all'):
        dataEvents = dataEvents[dataEvents['group'].isin(group)]
    # Data Cleaning
    dataEvents['time'] = pd.to_datetime(dataEvents['time'])
    dataEvents = dataEvents.sort_values('time') 

    newDataEvents = []
    #Select puzzle and actions
    notSelectedEvents = ['ws-mode_change', 'ws-click_nothing', 'ws-click_disabled', 'ws-select_shape', 'ws-deselect_shape', 'ws-paint', 'ws-palette_change', 'ws-toggle_paint_display', 'ws-toggle_snapshot_display', 'ws-create_user', 'ws-redo_action', 'ws-undo_action', 'ws-restart_puzzle', 'ws-puzzle_started']

    eventsWithMetaData = ['ws-create_shape', 'ws-delete_shape', 'ws-rotate_shape', 'ws-scale_shape', 'ws-move_shape']
    for user in dataEvents['group_user_id'].unique():
            #Select rows
            user_events = dataEvents[dataEvents['group_user_id'] == user]
            user_events_na_dropped = user_events.dropna()
            activePuzzle = None
            nAttempt = 1
            prevCheck = False
            prevEvent = None
            figureDict = dict()
            for enum, event in user_events_na_dropped.iterrows():
                #Ignore event
                if (prevCheck == True):
                    if (event['type'] == 'ws-puzzle_complete'):
                        prevEvent['metadata']['correct'] = True
                        newDataEvents.append(prevEvent)
                        prevCheck = False
                        prevEvent = None
                        continue
                    else:
                        prevEvent['metadata']['correct'] = False
                    newDataEvents.append(prevEvent)
                    prevCheck = False
                    prevEvent = None
                        
                if (event['type'] in notSelectedEvents):
                        continue

                elif(event['type'] == 'ws-start_level'):
                    activePuzzle = json.loads(event['data'])['task_id']
                    event['task_id'] = activePuzzle
                    #nAttempt += 1
                    if (event['task_id'] == 'Sandbox'):
                        continue
                    event['n_attempt'] = nAttempt
                    event['metadata'] = dict()
                    newDataEvents.append(event)
                
                elif (event['type'] == 'ws-create_shape'):
                    event['task_id'] = activePuzzle
                    if (event['task_id'] == 'Sandbox'):
                        continue
                    event['n_attempt'] = nAttempt
                    shape_id = json.loads(event['data'])['objSerialization']
                    shape_type = json.loads(event['data'])['shapeType']
                    figureDict[shape_id] = shape_type
                    event['metadata'] = dict()
                    event['metadata']['shape_id'] = shape_id 
                    event['metadata']['shape_type'] = shape_type 
                    newDataEvents.append(event)

                elif (event['type'] == 'ws-delete_shape' or event['type'] == 'ws-move_shape'):
                    event['task_id'] = activePuzzle
                    if (event['task_id'] == 'Sandbox'):
                        continue
                    event['n_attempt'] = nAttempt
                    if (event['type'] == 'ws-delete_shape'):
                        idList = json.loads(event['data'])['deletedShapes']
                    elif (event['type'] == 'ws-move_shape'):
                        idList = json.loads(event['data'])['selectedObjects']
                    for shapeId in idList:
                        shape_id = shapeId
                        shape_type = figureDict.get(shape_id)
                        if (shape_type == None):
                            figureDict[shape_id] = shape_type
                        event['metadata'] = dict()
                        event['metadata']['shape_id'] = shape_id 
                        event['metadata']['shape_type'] = shape_type 
                        newDataEvents.append(event)

                elif (event['type'] == 'ws-rotate_shape' or event['type'] == 'ws-scale_shape'):
                    event['task_id'] = activePuzzle
                    if (event['task_id'] == 'Sandbox'):
                        continue
                    event['n_attempt'] = nAttempt
                    shape_id = json.loads(event['data'])['selectedObject']
                    shape_type = figureDict.get(shape_id)
                    if (shape_type == None):
                        figureDict[shape_id] = shape_type
                    event['metadata'] = dict()
                    event['metadata']['shape_id'] = shape_id 
                    event['metadata']['shape_type'] = shape_type 
                    newDataEvents.append(event)

                elif(event['type'] in ['ws-exit_to_menu', 'ws-disconnect', 'ws-login_user']):
                    #figureDict.clear()
                    if (activePuzzle == 'Sandbox'):
                        continue
                    nAttempt +=1
   
                else :
                    event['task_id'] = activePuzzle
                    if (event['task_id'] == 'Sandbox'):
                        continue
                    event['n_attempt'] = nAttempt
                    event['metadata'] = dict()
                    if (event['type'] == 'ws-check_solution'):
                        prevCheck = True
                        prevEvent = event
                    else:
                        newDataEvents.append(event)
                        
    taskDf = pd.DataFrame(newDataEvents, columns=['id', 'time', 'group_user_id', 'task_id', 'n_attempt', 'type', 'metadata']) 

    data = taskDf
                        
    listEvent = ['ws-rotate_view', 'ws-rotate_shape', 'ws-undo_action', 'ws-move_shape', 'ws-snapshot', 'ws-scale_shape']
    
    #Collapse events
    dataConvert2 = []
    for user in data['group_user_id'].unique():
        individualDf = data[data['group_user_id'] == user]
        #Current action set
        currentAction = []
        #String with action types
        actionString = ""
        actualEvent = 'None'
        for enum, event in individualDf.iterrows():
            key = event['group_user_id']
            key_split = key.split('~')
            event['group_id'] = key_split[0]
            event['user'] = key_split[1]
            actualEvent = event['type']
            eq = True
            for a in currentAction:
                if (a['type'] != actualEvent):
                    #Ver si podemos compactar
                    eq = False
                    
            if (eq == False):      
                igual = True
                prev = ""
                for a2 in currentAction:
                    if (a2['type'] != prev):
                        if (prev == "") :
                            igual = True
                        else:
                            igual = False
                    prev = a2['type']
                if ((igual == True) and (prev in listEvent)):
                    add = currentAction[0]
                    #add['type'] = add['type'] + 'x' + str(len(currentAction))
                    add['n_times'] = dict()
                    add['n_times'][add['type']] = len(currentAction)
                    dataConvert2.append(add)
                    currentAction.clear()
                    currentAction.append(event)     
                else: #igual != True 
                    for a in currentAction:
                        a['n_times'] = dict()
                        a['n_times'][a['type']] = 1
                        dataConvert2.append(a)
                    currentAction.clear()
                    currentAction.append(event)
            else: #eq = True
                if (event['type'] not in listEvent):
                    currentAction.append(event)
                    for a in currentAction:
                        a['n_times'] = dict()
                        a['n_times'][a['type']] = 1
                        dataConvert2.append(a)
                    currentAction.clear()
                    
                else:
                    if (len(currentAction) > 0):
                            if (currentAction[0]['type'] in eventsWithMetaData):
                                #Event with metadata, check if it is the same shape_id
                                if (currentAction[0]['metadata']['shape_id'] == event['metadata']['shape_id']):
                                    currentAction.append(event)
                                else:
                                    add = currentAction[0]
                                    #add['type'] = add['type'] + 'x' + str(len(currentAction))
                                    add['n_times'] = dict()
                                    add['n_times'][add['type']] = len(currentAction)
                                    dataConvert2.append(add)
                                    currentAction.clear()
                                    currentAction.append(event)
                            #Event without metaData, just concatenate.
                            else:
                                currentAction.append(event) 

                    elif (len(currentAction) == 0):
                        currentAction.append(event)
                
                    
        #Add last elems
        #We must check if last elems can be also replaced.
        final = ""
        if (len(currentAction) > 0):
            igual2 = True
            prev = ""
            for a2 in currentAction:
                if (a2['type'] != prev):
                    if (prev == "") :
                        igual2 = True
                    else:
                        igual2 = False
                prev = a2['type']
            if ((igual == True) and (prev in listEvent)):
                add = currentAction[0]
                add['n_times'] = dict()
                add['n_times'][add['type']] = len(currentAction)
                dataConvert2.append(add)
                currentAction.clear()
                currentAction.append(event)     
            else: #igual != True 
                for a in currentAction:
                    a['n_times'] = dict()
                    a['n_times'][a['type']] = 1
                    dataConvert2.append(a)
                currentAction.clear()
                currentAction.append(event)
               
    #Create dataframe from list
    #consecutiveDf = pd.DataFrame(dataConvert2, columns=['id', 'time', 'group_user_id', 'task_id', 'n_attempt', 'type', 'metadata'])         
    data = pd.DataFrame(dataConvert2, columns=['group_id', 'user', 'task_id', 'n_attempt', 'type', 'n_times', 'metadata']) 
    
    #Recalculate n_attempt
    mod = []
    for user in data['user'].unique():
            previousAttempt = 1
            n_attempt = 1
            individualDf = data[data['user'] == user]
            for enum, event in individualDf.iterrows():
                if (event['n_attempt'] != previousAttempt):
                    n_attempt += 1
                previousAttempt = event['n_attempt']
                event['n_attempt'] = n_attempt
                mod.append(event)
    modDf = pd.DataFrame(mod, columns=['group_id', 'user', 'task_id', 'n_attempt', 'type', 'n_times', 'metadata'])
    return modDf

In [34]:
df = sequenceWithinPuzzles(dataEvents)
df

,group_id,user,task_id,n_attempt,type,n_times,metadata
405309,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-start_level,{'ws-start_level': 1},{}
405312,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-start_level,{'ws-start_level': 1},{}
405314,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-start_level,{'ws-start_level': 1},{}
405317,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-create_shape,{'ws-create_shape': 1},"{'shape_id': 1, 'shape_type': 1}"
405319,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-check_solution,{'ws-check_solution': 1},{'correct': True}
...,...,...,...,...,...,...,...
398574,e21640b4aea9349ad77d86d6017cb061,5cce64bf55190c8816d80129f875b351,1. One Box,1,ws-check_solution,{'ws-check_solution': 1},{'correct': True}
398614,e21640b4aea9349ad77d86d6017cb061,5cce64bf55190c8816d80129f875b351,2. Separated Boxes,2,ws-start_level,{'ws-start_level': 1},{}
398725,e21640b4aea9349ad77d86d6017cb061,5cce64bf55190c8816d80129f875b351,2. Separated Boxes,3,ws-start_level,{'ws-start_level': 1},{}
398859,e21640b4aea9349ad77d86d6017cb061,5cce64bf55190c8816d80129f875b351,2. Separated Boxes,3,ws-rotate_view,{'ws-rotate_view': 3},{}


In [35]:
#Pasamos a forma tabular para R
rEvents = ['ws-start_level', 'ws-create_shape', 'ws-move_shape', 'ws-rotate_shape', 'ws-scale_shape', 'ws-delete_shape', 'ws-check_solution', 'ws-rotate_view', 'ws-snapshot']
rList = []
for enum, event in df.iterrows():
    if event['type'] not in rEvents:
        continue
    event['n_times'] = event['n_times'][event['type']]
    if len(event['metadata']) == 2:
        event['shape_type'] = str(event['metadata']['shape_type'])
        event['shape_id'] = str(event['metadata']['shape_id'])
        
    else:
        event['shape_type'] = "-"
        event['shape_id'] = "-"     
    if len(event['metadata']) == 1:
        event['correct'] = event['metadata']['correct']
    else:
        event['correct'] = "-"
    rList.append(event)

rDf = pd.DataFrame(rList, columns = ['group_id', 'user', 'task_id', 'n_attempt', 'type', 'n_times', 'shape_id', 'shape_type', 'correct'])
rDf   
        

,group_id,user,task_id,n_attempt,type,n_times,shape_id,shape_type,correct
405309,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-start_level,1,-,-,-
405312,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-start_level,1,-,-,-
405314,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-start_level,1,-,-,-
405317,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-create_shape,1,1,1,-
405319,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-check_solution,1,-,-,True
...,...,...,...,...,...,...,...,...,...
398574,e21640b4aea9349ad77d86d6017cb061,5cce64bf55190c8816d80129f875b351,1. One Box,1,ws-check_solution,1,-,-,True
398614,e21640b4aea9349ad77d86d6017cb061,5cce64bf55190c8816d80129f875b351,2. Separated Boxes,2,ws-start_level,1,-,-,-
398725,e21640b4aea9349ad77d86d6017cb061,5cce64bf55190c8816d80129f875b351,2. Separated Boxes,3,ws-start_level,1,-,-,-
398859,e21640b4aea9349ad77d86d6017cb061,5cce64bf55190c8816d80129f875b351,2. Separated Boxes,3,ws-rotate_view,3,-,-,-


In [30]:
rDf.to_csv("seqWPOutput.csv", decimal = ".") 

In [10]:
#Segunda capa : compactamos eventos de manipulacion de una misma figura.
def collapseManipulationEvents (dataFrame):
    # ahora vamos a compactar acciones de 3 subacciones
    df = dataFrame
    eventsWithMetaData = ['ws-create_shape', 'ws-delete_shape', 'ws-rotate_shape', 'ws-scale_shape', 'ws-move_shape']
    dataConvert = []
    for user in df['user'].unique():
        individualDf = df[df['user'] == user]
        currentAction = []
        for enum, event in individualDf.iterrows():
            currentAction.append(event)
            if (len(currentAction) == 3):
                #Intentamos etiquetar la accion
                actionString = ""
                for action in currentAction:
                    actionString = actionString + action['type'] + " "
                if ((currentAction[0]['type'] in eventsWithMetaData and currentAction[1]['type'] in eventsWithMetaData and currentAction[2]['type'] in eventsWithMetaData) and (currentAction[0]['type'] != currentAction[1]['type'] and currentAction[1]['type'] != currentAction[2]['type']) and
                    (currentAction[0]['metadata']['shape_id'] == currentAction[1]['metadata']['shape_id'] and currentAction[0]['metadata']['shape_id'] == currentAction[2]['metadata']['shape_id'])) :
                    accion = currentAction[0]
                    nVeces = currentAction[0]['n_times']
                    for action in currentAction[1]['n_times'].keys():
                        nVeces[action] = currentAction[1]['n_times'][action]
                    for action in currentAction[2]['n_times'].keys():
                        if action in nVeces.keys():
                            action2 = action + "(2)"
                            nVeces[action2] = currentAction[2]['n_times'][action]
                        else:
                            nVeces[action] = currentAction[2]['n_times'][action]
                    nameString = 'ws-'
                    sub = currentAction[0]['type'].split("-")
                    action1 = sub[1].split("_")[0]
                    sub = currentAction[1]['type'].split("-")
                    action2 = sub[1].split("_")[0]
                    sub = currentAction[2]['type'].split("-")
                    action3 = sub[1].split("_")[0]
                    nameString = nameString + action1 + "_" + action2 + "_" + action3 +"_shape"
                    accion['type'] = nameString
                    accion['ntimes'] = nVeces
                    dataConvert.append(accion)
                    currentAction.clear()        
                else:
                    elem = currentAction.pop(0)
                    dataConvert.append(elem)
        #Añadir los ultimos elementos.
        for e in currentAction :
            dataConvert.append(e)
                #Limpiamos la lista
                #currentAction.clear()

    newDfConvert = pd.DataFrame(dataConvert, columns=['group_id', 'user', 'task_id', 'n_attempt', 'type', 'n_times', 'metadata'])
     
    # ahora vamos a compactar acciones de 2 subacciones
    dataConvert = []
    for user in newDfConvert['user'].unique():
        individualDf = newDfConvert[newDfConvert['user'] == user]
        currentAction = []
        for enum, event in individualDf.iterrows():
            currentAction.append(event)
            if (len(currentAction) == 2):
                #Intentamos etiquetar la accion
                actionString = ""
                for action in currentAction:
                    actionString = actionString + action['type'] + " "

                if ((currentAction[0]['type'] in eventsWithMetaData and currentAction[1]['type'] in eventsWithMetaData) and (currentAction[0]['type'] != currentAction[1]['type']) and
                    (currentAction[0]['metadata']['shape_id'] == currentAction[1]['metadata']['shape_id'])) :
                    accion = currentAction[0]
                    nVeces = currentAction[0]['n_times']
                    for action in currentAction[1]['n_times'].keys():
                        nVeces[action] = currentAction[1]['n_times'][action]
                    nameString = 'ws-'
                    sub = currentAction[0]['type'].split("-")
                    action1 = sub[1].split("_")[0]
                    sub = currentAction[1]['type'].split("-")
                    action2 = sub[1].split("_")[0]
                    nameString = nameString + action1 + "_" + action2 + "_shape"
                    accion['type'] = nameString
                    accion['ntimes'] = nVeces
                    dataConvert.append(accion)
                    currentAction.clear()

                else:
                    elem = currentAction.pop(0)
                    dataConvert.append(elem)
        #Añadir los ultimos elementos.
        for e in currentAction :
            dataConvert.append(e)
                #Limpiamos la lista
                #currentAction.clear()

    newDfConvert = pd.DataFrame(dataConvert, columns=['group_id', 'user', 'task_id', 'n_attempt', 'type', 'n_times', 'metadata'])
    
    return newDfConvert           


In [11]:
newDfConvert = collapseManipulationEvents(dataFrame = df)
newDfConvert

,group_id,user,task_id,n_attempt,type,n_times,metadata
405309,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-start_level,{'ws-start_level': 1},{}
405312,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-start_level,{'ws-start_level': 1},{}
405314,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-start_level,{'ws-start_level': 1},{}
405317,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-create_shape,{'ws-create_shape': 1},"{'shape_id': 1, 'shape_type': 1}"
405319,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-check_solution,{'ws-check_solution': 1},{'correct': True}
...,...,...,...,...,...,...,...
398574,e21640b4aea9349ad77d86d6017cb061,5cce64bf55190c8816d80129f875b351,1. One Box,1,ws-check_solution,{'ws-check_solution': 1},{'correct': True}
398614,e21640b4aea9349ad77d86d6017cb061,5cce64bf55190c8816d80129f875b351,2. Separated Boxes,2,ws-start_level,{'ws-start_level': 1},{}
398725,e21640b4aea9349ad77d86d6017cb061,5cce64bf55190c8816d80129f875b351,2. Separated Boxes,3,ws-start_level,{'ws-start_level': 1},{}
398859,e21640b4aea9349ad77d86d6017cb061,5cce64bf55190c8816d80129f875b351,2. Separated Boxes,3,ws-rotate_view,{'ws-rotate_view': 3},{}


In [143]:
sum = 0
for enum, event in newDfConvert.iterrows():
    for action in event['n_times'].keys():
        sum += event['n_times'][action]
sum

155861

In [12]:
def finalActions (dataFrame):
    data = dataFrame
    listEvent = ['ws-create_shape', 'ws-rotate_shape', 'ws-delete_shape', 'ws-move_shape', 'ws-scale_shape']
    dataConvert2 = []
    for user in data['user'].unique():
        individualDf = data[data['user'] == user]
        #Current action set
        currentAction = []
        #String with action types
        actionString = ""
        actualEvent = 'None'
        for enum, event in individualDf.iterrows():
            actualEvent = event['type']
            eq = True
            for a in currentAction:
                if (a['type'] != actualEvent):
                    #Ver si podemos compactar
                    eq = False
                    
            if (eq == False):      
                igual = True
                prev = ""
                for a2 in currentAction:
                    if (a2['type'] != prev):
                        if (prev == "") :
                            igual = True
                        else:
                            igual = False
                    prev = a2['type']
                if ((igual == True) and (prev in listEvent)):
                    if (len(currentAction) > 1):
                        add = currentAction[0]
                        nVeces = dict()
                        i = 1
                        metadata = dict()
                        for action in currentAction:
                            aName = action['type'] + "(" + str(i) + ")"
                            nVeces[aName] = action['n_times'][action['type']]
                            metadata[aName] = action['metadata']
                            i += 1
                        name = add['type'] + "s"
                        add['type'] = name
                        add['n_times'] = nVeces
                        add['metadata'] = metadata    
                        dataConvert2.append(add)
                        currentAction.clear()
                        currentAction.append(event)  
                    else:
                        dataConvert2.append(currentAction[0])
                        currentAction.clear()
                        currentAction.append(event)
                else: #igual != True 
                    for a in currentAction:
                        dataConvert2.append(a)
                    currentAction.clear()
                    currentAction.append(event)
            else: #eq = True
                if (event['type'] not in listEvent):
                    currentAction.append(event)
                    for a in currentAction:
                        dataConvert2.append(a)
                    currentAction.clear()
                    
                else:
                    currentAction.append(event)
                
                    
        #Add last elems
        #We must check if last elems can be also replaced.
        final = ""
        if (len(currentAction) > 0):
            igual2 = True
            prev = ""
            for a2 in currentAction:
                if (a2['type'] != prev):
                    if (prev == "") :
                        igual2 = True
                    else:
                        igual2 = False
                prev = a2['type']
            if ((igual == True) and (prev in listEvent)):
                if (len(currentAction) > 1):
                    add = currentAction[0]
                    nVeces = dict()
                    i = 1
                    metadata = dict()
                    for action in currentAction:
                        aName = action['type'] + "(" + str(i) + ")"
                        nVeces[aName] = action['n_times'][action['type']]
                        metadata[aName] = action['metadata']
                        i += 1
                    name = add['type'] + "s"
                    add['type'] = name
                    add['n_times'] = nVeces
                    add['metadata'] = metadata    
                    dataConvert2.append(add)
                    currentAction.clear()
                    currentAction.append(event)  
                else:
                    dataConvert2.append(currentAction[0])
                    currentAction.clear()
                    currentAction.append(event)
                    
            else: #igual != True 
                for a in currentAction:
                    dataConvert2.append(a)
                currentAction.clear()
                currentAction.append(event)
               
    #Create dataframe from list
    #consecutiveDf = pd.DataFrame(dataConvert2, columns=['id', 'time', 'group_user_id', 'task_id', 'n_attempt', 'type', 'metadata'])         
    data = pd.DataFrame(dataConvert2, columns=['group_id', 'user', 'task_id', 'n_attempt', 'type', 'n_times', 'metadata']) 
    
    #Ahora vamos a definir nuevas acciones de más alto nivel.
    #3-gramas y 2-gramas, son los mas comunes.        
    dataConvert = []
    for user in data['user'].unique():
        individualDf = data[data['user'] == user]
        currentAction = []
        for enum, event in individualDf.iterrows():
            currentAction.append(event)
            if (len(currentAction) == 3):
                #Intentamos etiquetar la accion
                actionString = ""
                for action in currentAction:
                    actionString = actionString + action['type'] + " "

                if (actionString == 'ws-rotate_view ws-move_shape ws-rotate_view '):
                        accion = currentAction[0]
                        nVeces = dict()
                        for ac in currentAction:
                            if ac['type'] not in nVeces.keys():
                                nVeces[ac['type']] = ac['n_times'][ac['type']]
                            else:
                                name2 = ac['type'] + "(2)"
                                nVeces[name2] = ac['n_times'][ac['type']]
                        metadata = currentAction[1]['metadata']
                        accion['type'] = 'ws-move_between_rotate_views'
                        accion['n_times'] = nVeces
                        accion['metadata'] = metadata
                        dataConvert.append(accion)
                        currentAction.clear()  
                elif ((actionString ==  'ws-move_shape ws-rotate_view ws-move_shape') and (currentAction[0]['metadata']['shape_id'] == currentAction[2]['metadata']['shape_id'])):
                        accion = currentAction[0]
                        nVeces = dict()
                        for ac in currentAction:
                            if ac['type'] not in nVeces.keys():
                                nVeces[ac['type']] = ac['n_times'][ac['type']]
                            else:
                                name2 = ac['type'] + "(2)"
                                nVeces[name2] = ac['n_times'][ac['type']]
                        metadata = currentAction[0]['metadata']
                        accion['type'] = 'ws-rotate_between_move_shape'
                        accion['n_times'] = nVeces
                        accion['metadata'] = metadata
                        dataConvert.append(accion)
                        currentAction.clear()  
                elif (actionString == 'ws-rotate_view ws-snapshot ws-rotate_view ' or actionString == 'ws-snapshot ws-rotate_view ws-snapshot '):
                        accion = currentAction[0]
                        nVeces = dict()
                        for ac in currentAction:
                            if ac['type'] not in nVeces.keys():
                                nVeces[ac['type']] = ac['n_times'][ac['type']]
                            else:
                                name2 = ac['type'] + "(2)"
                                nVeces[name2] = ac['n_times'][ac['type']]
                        metadata = currentAction[0]['metadata']
                        if (actionString == 'ws-rotate_view ws-snapshot ws-rotate_view '):
                            accion['type'] = 'ws-snapshot_between_rotate_views'
                        else:
                            accion['type'] = 'ws-rotate_view_between_snapshots'
                        accion['n_times'] = nVeces
                        accion['metadata'] = metadata
                        dataConvert.append(accion)
                        currentAction.clear() 
                elif (actionString == 'ws-rotate_view ws-move_shape ws-check_solution '):
                        accion = currentAction[0]
                        nVeces = dict()
                        for ac in currentAction:
                            if ac['type'] not in nVeces.keys():
                                nVeces[ac['type']] = ac['n_times'][ac['type']]
                            else:
                                name2 = ac['type'] + "(2)"
                                nVeces[name2] = ac['n_times'][ac['type']]
                        metadata = currentAction[1]['metadata']
                        metadata['correct'] = currentAction[2]['metadata']['correct']
                        accion['type'] = 'ws-rotate_view_move_check'                   
                        accion['n_times'] = nVeces
                        accion['metadata'] = metadata
                        dataConvert.append(accion)
                        currentAction.clear()
                elif (actionString == 'ws-move_shape ws-rotate_view ws-check_solution '):
                        accion = currentAction[0]
                        nVeces = dict()
                        for ac in currentAction:
                            if ac['type'] not in nVeces.keys():
                                nVeces[ac['type']] = ac['n_times'][ac['type']]
                            else:
                                name2 = ac['type'] + "(2)"
                                nVeces[name2] = ac['n_times'][ac['type']]
                        metadata = currentAction[0]['metadata']
                        metadata['correct'] = currentAction[2]['metadata']['correct']
                        accion['type'] = 'ws-move_rotate_view_check'                   
                        accion['n_times'] = nVeces
                        accion['metadata'] = metadata
                        dataConvert.append(accion)
                        currentAction.clear()    
                elif (actionString == 'ws-snapshot ws-move_shape ws-snapshot ' or actionString == 'ws-snapshot ws-rotate_shape ws-snapshot ' or actionString == 'ws-snapshot ws-scale_shape ws-snapshot '):
                        accion = currentAction[0]
                        nVeces = dict()
                        for ac in currentAction:
                            if ac['type'] not in nVeces.keys():
                                nVeces[ac['type']] = ac['n_times'][ac['type']]
                            else:
                                name2 = ac['type'] + "(2)"
                                nVeces[name2] = ac['n_times'][ac['type']]
                        metadata = currentAction[1]['metadata']
                        sub = currentAction[1]['type'].split("-")
                        action1 = sub[1].split("_")[0]
                        accion['type'] = 'ws-' + action1 + '_shape_between_snapshots'                  
                        accion['n_times'] = nVeces
                        accion['metadata'] = metadata
                        dataConvert.append(accion)
                        currentAction.clear()
                elif (actionString == 'ws-check_solution ws-move_shape ws-check_solution ' or actionString == 'ws-check_solution ws-rotate_shape ws-check_solution ' or actionString == 'ws-check_solution ws-scale_shape ws-check_solution '):
                        accion = currentAction[0]
                        nVeces = dict()
                        for ac in currentAction:
                            if ac['type'] not in nVeces.keys():
                                nVeces[ac['type']] = ac['n_times'][ac['type']]
                            else:
                                name2 = ac['type'] + "(2)"
                                nVeces[name2] = ac['n_times'][ac['type']]
                        metadata = currentAction[1]['metadata']
                        metadata['correct'] = currentAction[0]['metadata']['correct']
                        metadata['correct(2)'] = currentAction[2]['metadata']['correct']
                        sub = currentAction[1]['type'].split("-")
                        action1 = sub[1].split("_")[0]
                        accion['type'] = 'ws-' + action1 + '_shape_between_checks'              
                        accion['n_times'] = nVeces
                        accion['metadata'] = metadata
                        dataConvert.append(accion)
                        currentAction.clear()



                else:
                    elem = currentAction.pop(0)
                    dataConvert.append(elem)
        #Añadir los ultimos elementos.
        for e in currentAction :
            dataConvert.append(e)
            #Limpiamos la lista
            #currentAction.clear()

    data = pd.DataFrame(dataConvert, columns=['group_id', 'user', 'task_id', 'n_attempt', 'type', 'n_times', 'metadata'])

    dataConvert = []
    for user in data['user'].unique():
        individualDf = data[data['user'] == user]
        currentAction = []
        for enum, event in individualDf.iterrows():
            currentAction.append(event)
            if (len(currentAction) == 2):
                #Intentamos etiquetar la accion
                actionString = ""
                for action in currentAction:
                    actionString = actionString + action['type'] + " "

                if (actionString == 'ws-check_solution ws-rotate_view '):
                        accion = currentAction[0]
                        nVeces = dict()
                        for ac in currentAction:
                            if ac['type'] not in nVeces.keys():
                                nVeces[ac['type']] = ac['n_times'][ac['type']]
                            else:
                                name2 = ac['type'] + "(2)"
                                nVeces[name2] = ac['n_times'][ac['type']]
                        accion['type'] = 'ws-check_and_rotate_view'             
                        accion['n_times'] = nVeces
                        dataConvert.append(accion)
                        currentAction.clear()
                elif (actionString == 'ws-move_shapes ws-check_solution ' or actionString == 'ws-move_shape ws-check_solution '):
                        accion = currentAction[0]
                        nVeces = currentAction[0]['n_times']
                        nVeces[currentAction[1]['type']] = currentAction[1]['n_times'][currentAction[1]['type']] 
                        if (actionString == 'ws-move_shapes ws-check_solution '):
                            accion['type'] = 'ws-move_shapes_and_check' 
                        else:
                            accion['type'] = 'ws-move_shape_and_check'
                        accion['metadata']['correct'] = currentAction[1]['metadata']['correct']
                        accion['n_times'] = nVeces
                        dataConvert.append(accion)
                        currentAction.clear()    
                else:
                    elem = currentAction.pop(0)
                    dataConvert.append(elem)
        #Añadir los ultimos elementos.
        for e in currentAction :
            dataConvert.append(e)

    data = pd.DataFrame(dataConvert, columns=['group_id', 'user', 'task_id', 'n_attempt', 'type', 'n_times', 'metadata'])
    return data



In [13]:
new = finalActions(dataFrame = newDfConvert)
new

,group_id,user,task_id,n_attempt,type,n_times,metadata
405309,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-start_level,{'ws-start_level': 1},{}
405312,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-start_level,{'ws-start_level': 1},{}
405314,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-start_level,{'ws-start_level': 1},{}
405317,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-create_shape,{'ws-create_shape': 1},"{'shape_id': 1, 'shape_type': 1}"
405319,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,1. One Box,1,ws-check_solution,{'ws-check_solution': 1},{'correct': True}
...,...,...,...,...,...,...,...
398574,e21640b4aea9349ad77d86d6017cb061,5cce64bf55190c8816d80129f875b351,1. One Box,1,ws-check_solution,{'ws-check_solution': 1},{'correct': True}
398614,e21640b4aea9349ad77d86d6017cb061,5cce64bf55190c8816d80129f875b351,2. Separated Boxes,2,ws-start_level,{'ws-start_level': 1},{}
398725,e21640b4aea9349ad77d86d6017cb061,5cce64bf55190c8816d80129f875b351,2. Separated Boxes,3,ws-start_level,{'ws-start_level': 1},{}
398859,e21640b4aea9349ad77d86d6017cb061,5cce64bf55190c8816d80129f875b351,2. Separated Boxes,3,ws-rotate_view,{'ws-rotate_view': 3},{}


In [15]:
from nltk import ngrams
#Vamos a ver las secuencias mas frecuentes de acciones usando n-gramas.
#Para ello nos centramos en cada puzzle por separado.

#Para los ngramas no tenemos en cuenta los tutoriales.
selectedPuzzles = ['Square Cross-Sections', 'Bird Fez', 'Pi Henge', '45-Degree Rotations',  'Pyramids are Strange', 'Boxes Obscure Spheres', 'Object Limits', 'Warm Up', 'Angled Silhouette',
                    'Sugar Cones','Stranger Shapes', 'Tall and Small', 'Ramp Up and Can It', 'More Than Meets Your Eye', 'Not Bird', 'Unnecesary', 'Zzz', 'Bull Market', 'Few Clues', 'Orange Dance', 'Bear Market']

data = newDfConvert
actionsDict = {}
for user in data['user'].unique():
    #Select rows
    individualDf = data[data['user'] == user]
    for attempt in individualDf['n_attempt'].unique():
        attemptDf = individualDf[individualDf['n_attempt'] == attempt]
        actions = ""
        for enum, event in attemptDf.iterrows() :
            if event['task_id'] not in selectedPuzzles :
                continue
            if user not in actionsDict.keys() :
                actionsDict[user] = dict()
            if attempt not in actionsDict[user].keys():
                actionsDict[user][attempt] = actions
            actionsDict[user][attempt] = actionsDict[user][attempt] + event['type'] + " "

In [16]:
pd.set_option('display.max_colwidth', 100)
sentence = ""
total = list()
finalCounter = dict()
for key in actionsDict.keys():
    for key2 in actionsDict[key].keys():
        sentence = actionsDict[key][key2]
        n = 2
        sixgrams = ngrams(sentence.split(), n)
        total.append(sixgrams)
for col in total :
    esBigramFreq = collections.Counter(col)
    for key in esBigramFreq.keys():
        if key not in finalCounter.keys():
            finalCounter[key] = esBigramFreq[key]
        else :
            finalCounter[key] = finalCounter[key] + esBigramFreq[key]
            
dfCounter = pd.DataFrame(finalCounter.items(), columns=['action', 'count'])
dfCounter.sort_values('count', ascending = False)[0:20]
    
    

,action,count
4,"(ws-move_shape, ws-move_shape)",5170
3,"(ws-rotate_view, ws-move_shape)",3396
16,"(ws-create_shape, ws-create_shape)",3344
39,"(ws-move_shape, ws-rotate_view)",2791
55,"(ws-move_shape, ws-check_solution)",1932
25,"(ws-check_solution, ws-rotate_view)",1923
42,"(ws-rotate_view, ws-snapshot)",1565
176,"(ws-move_shape, ws-snapshot)",1507
18,"(ws-start_level, ws-create_shape)",1409
172,"(ws-snapshot, ws-move_shape)",1312


In [120]:
def studentSequence(studentId, dataFrame):
    eventsWithShapeData = ['ws-create_shape', 'ws-rotate_shape', 'ws-delete_shape', 'ws-scale_shape', 'ws-move_shape']
    shapeTypes = {'1': 'cube', '2': 'pyramid', '3': 'ramp', '4': 'cylinder', '5': 'cone', '6': 'sphere'}
    eventNames = {'ws-create_shape' : 'Create', 'ws-rotate_shape' : 'Rotate', 'ws-delete_shape': "Delete", 'ws-scale_shape' : 'Scale',
                 'ws-move_shape' : 'Move', 'ws-check_solution' : "Submit", 'ws-snapshot' : "Snapshot", 'ws-undo_action': 'Undo', 
                 'ws-rotate_view' : 'Rotate View', 'ws-restart_puzzle' : 'Restart', 'ws-redo_action' : 'Redo'}
    studentEvents = dataFrame[dataFrame['user'] == studentId]
    sequence = ""
    for puzzle in studentEvents['n_attempt'].unique():
        attempt = False
        attemptEvents = studentEvents[studentEvents['n_attempt'] == puzzle]
        for enum, event in attemptEvents.iterrows():
            if (attempt == False):
                sequence = sequence[0:len(sequence)-2]
                sequence = sequence + "\n" + str(event['n_attempt']) + " - " + event['task_id'] + ':\n'
                attempt = True
              
            if event['type'] in eventsWithShapeData:
                sequence = sequence +  eventNames[event['type']] + " " + shapeTypes[str(event['metadata']['shape_type'])] + ": " + str(event['metadata']['shape_id']) 
                if (event['n_times'][event['type']] > 1):
                    sequence = sequence + " (x" + str(event['n_times'][event['type']]) + ")"
                sequence = sequence + " - "
            elif (event['type'] == 'ws-check_solution'):
                sequence = sequence + eventNames[event['type']] + " ("
                if (event['metadata']['correct'] == True):
                    sequence = sequence + "Correct"
                else:
                    sequence = sequence + "Incorrect"
                sequence = sequence + ")" + " \n "     
            elif (event['type'] in eventNames.keys()):
                sequence = sequence + eventNames[event['type']]
                if (event['n_times'][event['type']] > 1):
                    sequence = sequence + " (x" + str(event['n_times'][event['type']]) + ")"
                sequence = sequence + " - "
    return sequence  

In [121]:
seq = studentSequence(studentId = '15246dfc4342ffe2601e4ecee9017eea', dataFrame = df)
print(seq)


1 - Square Cross-Sections:
Create pyramid: 1 - Create cylinder: 2 - Create pyramid: 3 - Snapshot (x2) - Rotate pyramid: 1 (x3) - Rotate pyramid: 3 - Snapshot - Rotate pyramid: 1 (x3) - Delete pyramid: 3 - Create ramp: 4 - Rotate ramp: 4 (x4) - Delete pyramid: 1 - Create ramp: 5 - Rotate ramp: 5 (x7) - Scale ramp: 5 (x3) - Undo (x3) - Move ramp: 5 (x8) - Move ramp: 4 - Snapshot (x2) - Create ramp: 6 - Delete ramp: 6 - Delete ramp: 5 - Create cone: 7 - Move cone: 7 - Snapshot - Move cone: 7 (x2) - Rotate cone: 7 - Rotate ramp: 4 - Rotate cylinder: 2 - Snapshot - Rotate ramp: 4 (x2) - Rotate View - Rotate ramp: 4 (x3) - Rotate cylinder: 2 - Rotate cone: 7 (x5) - Rotate cylinder: 2 (x2) - Rotate cone: 7 (x9) 
2 - Bird Fez:
Create sphere: 1 - Create cube: 2 - Scale cube: 2 - Undo - Scale sphere: 1 (x2) - Create cone: 3 - Scale cone: 3 - Rotate cone: 3 (x2) - Move cone: 3 (x2) - Scale cone: 3 (x3) - Undo (x3) - Move cone: 3 (x5) - Snapshot - Move cube: 2 - Submit (Incorrect) 
 Rotate View -